#1. Load and Explore the Data
- Import the necessary libraries: pandas and sqlite3.

- Connect to the IPL database and load the master table to understand the structure.


In [10]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3
from google.colab import files

# Upload the SQLite database file
uploaded = files.upload()

# Connect to the database (file will be in /content)
conn = sqlite3.connect('database_Daily Challenge (1).sqlite')

Saving database.sqlite to database (1).sqlite


- Load all the tables and print their column names to identify common columns.

In [11]:
import sqlite3, pandas as pd
from IPython.display import display

db_path = "/content/database.sqlite"   # adjust if needed
conn = sqlite3.connect(db_path)

# Master (tables/views)
master = pd.read_sql("""
  SELECT name AS tbl_name, type, sql
  FROM sqlite_master
  WHERE type IN ('table','view')
  ORDER BY type, name;
""", conn)
display(master)

# List columns for each user table
tables = pd.read_sql("""
  SELECT name
  FROM sqlite_master
  WHERE type='table' AND name NOT LIKE 'sqlite_%'
  ORDER BY name;
""", conn)['name'].tolist()

for t in tables:
    cols = pd.read_sql(f'PRAGMA table_info("{t}")', conn)
    print(f"\n{t}: {[c for c in cols.name]}")


,tbl_name,type,sql
0,Ball_by_Ball,table,CREATE TABLE [Ball_by_Ball] (\n\t[Match_Id]\ti...
1,Batsman_Scored,table,CREATE TABLE [Batsman_Scored] (\n\t[Match_Id]\...
2,Batting_Style,table,CREATE TABLE [Batting_Style] (\n\t[Batting_Id]...
3,Bowling_Style,table,CREATE TABLE [Bowling_Style] (\n\t[Bowling_Id]...
4,City,table,CREATE TABLE [City] (\n\t[City_Id]\tinteger NO...
5,Country,table,CREATE TABLE [Country] (\n\t[Country_Id]\tinte...
6,Extra_Runs,table,CREATE TABLE [Extra_Runs] (\n\t[Match_Id]\tint...
7,Extra_Type,table,CREATE TABLE [Extra_Type] (\n\t[Extra_Id]\tint...
8,Match,table,CREATE TABLE [Match] (\n\t[Match_Id]\tinteger ...
9,Out_Type,table,CREATE TABLE [Out_Type] (\n\t[Out_Id]\tinteger...



Ball_by_Ball: ['Match_Id', 'Over_Id', 'Ball_Id', 'Innings_No', 'Team_Batting', 'Team_Bowling', 'Striker_Batting_Position', 'Striker', 'Non_Striker', 'Bowler']

Batsman_Scored: ['Match_Id', 'Over_Id', 'Ball_Id', 'Runs_Scored', 'Innings_No']

Batting_Style: ['Batting_Id', 'Batting_hand']

Bowling_Style: ['Bowling_Id', 'Bowling_skill']

City: ['City_Id', 'City_Name', 'Country_id']

Country: ['Country_Id', 'Country_Name']

Extra_Runs: ['Match_Id', 'Over_Id', 'Ball_Id', 'Extra_Type_Id', 'Extra_Runs', 'Innings_No']

Extra_Type: ['Extra_Id', 'Extra_Name']

Match: ['Match_Id', 'Team_1', 'Team_2', 'Match_Date', 'Season_Id', 'Venue_Id', 'Toss_Winner', 'Toss_Decide', 'Win_Type', 'Win_Margin', 'Outcome_type', 'Match_Winner', 'Man_of_the_Match']

Out_Type: ['Out_Id', 'Out_Name']

Outcome: ['Outcome_Id', 'Outcome_Type']

Player: ['Player_Id', 'Player_Name', 'DOB', 'Batting_hand', 'Bowling_skill', 'Country_Name']

Player_Match: ['Match_Id', 'Player_Id', 'Role_Id', 'Team_Id']

Rolee: ['Role_Id', 'Rol

# 2. Query 1: Select All Columns from Player’s Table

Write and execute a SQL query to select all columns from the Player_Match table.

In [12]:
q1 = "SELECT * FROM Player_Match;"
df_q1 = pd.read_sql(q1, conn)
display(df_q1.head(10))
df_q1.to_csv("/content/Q1_Player_Match_all_columns.csv", index=False)


,Match_Id,Player_Id,Role_Id,Team_Id
0,335987,1,1,1
1,335987,2,3,1
2,335987,3,3,1
3,335987,4,3,1
4,335987,5,3,1
5,335987,6,1,2
6,335987,7,3,2
7,335987,8,3,2
8,335987,9,3,2
9,335987,10,3,2


# 3. Query 2: Batsman vs Runs

Write and execute a SQL query to calculate the **total runs scored by each batsman**.

In [14]:
q2 = '''SELECT p.Player_Id, p.Player_Name,
     SUM(bs.Runs_Scored) AS total_runs
     FROM Batsman_Scored AS bs
     JOIN Ball_by_Ball AS bb
     ON bs.Match_Id = bb.Match_Id
     AND bs.Over_Id  = bb.Over_Id
     AND bs.Ball_Id  = bb.Ball_Id
     AND bs.Innings_No = bb.Innings_No
     JOIN Player AS p
     ON p.Player_Id = bb.Striker
     GROUP BY p.Player_Id, p.Player_Name
     ORDER BY total_runs DESC;''';

df_q2 = pd.read_sql(q2, conn)
display(df_q2.head(10))
df_q2.to_csv("/content/Q2_Total_runs_by_batsman.csv", index=False)

,Player_Id,Player_Name,total_runs
0,21,SK Raina,4106
1,8,V Kohli,4105
2,57,RG Sharma,3874
3,40,G Gambhir,3634
4,162,CH Gayle,3447
5,46,RV Uthappa,3390
6,187,DA Warner,3373
7,20,MS Dhoni,3270
8,110,AB de Villiers,3270
9,42,S Dhawan,3082


# 4. Query 3: Fifties and Hundreds

Write and execute a SQL query to calculate the **number of fifties and hundreds scored by each batsman**.

In [20]:
q3 = '''
WITH PlayerInningsScores AS (
    SELECT
        bb.Match_Id,
        bb.Innings_No,
        p.Player_Id,
        p.Player_Name,
        SUM(bs.Runs_Scored) AS total_runs_in_innings
    FROM Ball_by_Ball AS bb
    JOIN Batsman_Scored AS bs
        ON bb.Match_Id = bs.Match_Id
        AND bb.Over_Id = bs.Over_Id
        AND bb.Ball_Id = bs.Ball_Id
        AND bb.Innings_No = bs.Innings_No
    JOIN Player AS p
        ON bb.Striker = p.Player_Id
    GROUP BY
        bb.Match_Id,
        bb.Innings_No,
        p.Player_Id,
        p.Player_Name
)

-- Now, using PlayerInningsScores, calculate fifties and hundreds
SELECT
    pis.Player_Id,
    pis.Player_Name,
    SUM(CASE WHEN pis.total_runs_in_innings >= 50 AND pis.total_runs_in_innings < 100 THEN 1 ELSE 0 END) AS fifties,
    SUM(CASE WHEN pis.total_runs_in_innings >= 100 THEN 1 ELSE 0 END) AS hundreds
FROM PlayerInningsScores AS pis
GROUP BY
    pis.Player_Id,
    pis.Player_Name
ORDER BY
    hundreds DESC, fifties DESC;
''';
                                                                 ##CASE is like an if-else inside SQL
df_q3 = pd.read_sql(q3, conn)                                    # Runs the SQL query and loads the result into a pandas DataFrame
display(df_q3.head(10))                                          # Displays the first 10 rows of the DataFrame
df_q3.to_csv("/content/Q3_Fifties_and_Hundreds_by_batsman.csv", index=False)        # Saves the result to a CSV file

,Player_Id,Player_Name,fifties,hundreds
0,162,CH Gayle,20,5
1,8,V Kohli,26,4
2,110,AB de Villiers,21,3
3,187,DA Warner,32,2
4,41,V Sehwag,16,2
5,32,SR Watson,14,2
6,185,M Vijay,13,2
7,53,AC Gilchrist,11,2
8,2,BB McCullum,11,2
9,57,RG Sharma,29,1


#5. Query 4: Best Bowling Figures

Write and execute a SQL query to find the best bowling figures for each bowler.

In [21]:
import pandas as pd

q4 = '''
SELECT
    p.Player_Name,
    COUNT(wt.Match_Id) AS total_wickets                     #link each ball bowled to whether it resulted in a wicket
FROM Ball_by_Ball AS bb
JOIN Wicket_Taken AS wt
    ON bb.Match_Id = wt.Match_Id
    AND bb.Over_Id = wt.Over_Id
    AND bb.Ball_Id = wt.Ball_Id
    AND bb.Innings_No = wt.Innings_No
JOIN Player AS p
    ON bb.Bowler = p.Player_Id
WHERE wt.Kind_Out NOT IN ('run out', 'retired hurt')
GROUP BY p.Player_Name
ORDER BY total_wickets DESC;
''';

df_q4 = pd.read_sql(q4, conn)
display(df_q4.head(10))
df_q4.to_csv("/content/Q4_Best_bowling_figures.csv", index=False)

,Player_Name,total_wickets
0,SL Malinga,159
1,DJ Bravo,137
2,A Mishra,132
3,Harbhajan Singh,128
4,PP Chawla,127
5,R Vinay Kumar,123
6,A Nehra,111
7,R Ashwin,110
8,Z Khan,107
9,RP Singh,100


#6. Query 5: Comprehensive Career Metrics

Combine all the previous chunks into a single comprehensive query to get detailed career metrics for players.


Combining several sub-queries (CTEs) to gather player information, total runs, fifties, hundreds, and total wickets. Start by defining the CTEs for total runs, innings scores, fifties/hundreds, and bowler wickets, and then join them with the Player table.



In [22]:
q5 = '''
WITH BatsmanScores AS (
    SELECT
        p.Player_Id,
        p.Player_Name,
        SUM(bs.Runs_Scored) AS total_runs
    FROM Player AS p
    JOIN Ball_by_Ball AS bb ON p.Player_Id = bb.Striker
    JOIN Batsman_Scored AS bs
        ON bb.Match_Id = bs.Match_Id
        AND bb.Over_Id = bs.Over_Id
        AND bb.Ball_Id = bs.Ball_Id
        AND bb.Innings_No = bs.Innings_No
    GROUP BY
        p.Player_Id, p.Player_Name
),
PlayerInningsScores AS (
    SELECT
        bb.Match_Id,
        bb.Innings_No,
        p.Player_Id,
        p.Player_Name,
        SUM(bs.Runs_Scored) AS total_runs_in_innings
    FROM Player AS p
    JOIN Ball_by_Ball AS bb ON p.Player_Id = bb.Striker
    JOIN Batsman_Scored AS bs
        ON bb.Match_Id = bs.Match_Id
        AND bb.Over_Id = bs.Over_Id
        AND bb.Ball_Id = bs.Ball_Id
        AND bb.Innings_No = bs.Innings_No
    GROUP BY
        bb.Match_Id,
        bb.Innings_No,
        p.Player_Id,
        p.Player_Name
),
FiftiesHundreds AS (
    SELECT
        pis.Player_Id,
        SUM(CASE WHEN pis.total_runs_in_innings >= 50 AND pis.total_runs_in_innings < 100 THEN 1 ELSE 0 END) AS fifties,
        SUM(CASE WHEN pis.total_runs_in_innings >= 100 THEN 1 ELSE 0 END) AS hundreds
    FROM PlayerInningsScores AS pis
    GROUP BY
        pis.Player_Id
),
BowlerWickets AS (
    SELECT
        p.Player_Id,
        COUNT(wt.Match_Id) AS total_wickets
    FROM Player AS p
    JOIN Ball_by_Ball AS bb ON p.Player_Id = bb.Bowler
    JOIN Wicket_Taken AS wt
        ON bb.Match_Id = wt.Match_Id
        AND bb.Over_Id = wt.Over_Id
        AND bb.Ball_Id = wt.Ball_Id
        AND bb.Innings_No = wt.Innings_No
    WHERE wt.Kind_Out NOT IN ('run out', 'retired hurt')
    GROUP BY
        p.Player_Id
)
SELECT
    p.Player_Id,
    p.Player_Name,
    COALESCE(bs.total_runs, 0) AS total_runs,
    COALESCE(fh.fifties, 0) AS fifties,
    COALESCE(fh.hundreds, 0) AS hundreds,
    COALESCE(bw.total_wickets, 0) AS total_wickets
FROM Player AS p
LEFT JOIN BatsmanScores AS bs ON p.Player_Id = bs.Player_Id
LEFT JOIN FiftiesHundreds AS fh ON p.Player_Id = fh.Player_Id
LEFT JOIN BowlerWickets AS bw ON p.Player_Id = bw.Player_Id
ORDER BY
    total_runs DESC, total_wickets DESC;
'''
df_q5 = pd.read_sql(q5, conn)
display(df_q5.head(10))
df_q5.to_csv("/content/Q5_Comprehensive_Career_Metrics.csv", index=False)

,Player_Id,Player_Name,total_runs,fifties,hundreds,total_wickets
0,21,SK Raina,4106,28,1,29
1,8,V Kohli,4105,26,4,5
2,57,RG Sharma,3874,29,1,16
3,40,G Gambhir,3634,31,0,0
4,162,CH Gayle,3447,20,5,19
5,46,RV Uthappa,3390,17,0,0
6,187,DA Warner,3373,32,2,0
7,20,MS Dhoni,3270,16,0,0
8,110,AB de Villiers,3270,21,3,0
9,42,S Dhawan,3082,25,0,4


## Summary:

### Q&A
The comprehensive career metrics now available for each player include their unique `Player_Id`, `Player_Name`, `total_runs` (cumulative runs scored across all innings), `fifties` (the count of innings where the player scored between 50 and 99 runs), `hundreds` (the count of innings where the player scored 100 runs or more), and `total_wickets` (the total number of wickets taken, excluding 'run out' and 'retired hurt' dismissals). This consolidates both batting and bowling performance into a single, comprehensive record for each player.

### Data Analysis Key Findings
*   The `df_q5` DataFrame successfully compiles comprehensive career metrics for all players, including `Player_Id`, `Player_Name`, `total_runs`, `fifties`, `hundreds`, and `total_wickets`.
*   The top run-scorer identified is SK Raina with 4106 runs, who also has 28 fifties, 1 hundred, and 29 wickets, indicating a strong all-round performance.
*   V Kohli closely follows with 4105 runs, notably having a higher number of centuries (4 hundreds) compared to fifties (26 fifties), alongside 5 wickets.
*   RG Sharma is another significant batting contributor with 3874 runs, 29 fifties, 1 hundred, and has also taken 16 wickets.

### Insights or Next Steps
*   The compiled comprehensive metrics allow for an in-depth analysis to identify top all-rounders (e.g., SK Raina's balanced contribution in runs and wickets) versus specialist batsmen (e.g., V Kohli's higher number of centuries) or bowlers.
*   This dataset can be further utilized for advanced player ranking systems, performance trend analysis over seasons, or to evaluate player value for team selection and scouting purposes.
